<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marco-canas/Machine-Learning/blob/main/ML/classes/class_march_3/class_march_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table>

# Predicción de calidad de vinos

In [ ]:
#Librerías para análisis preliminar
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
vino=pd.read_csv("winequality-red_grupo_6_castrillon.csv") #Dataset

# Trabajo de regresión 

### Integrantes grupo seis

* Andrés Orlando Castrillón López **Doc**: 1036646831
* Davison Andrés Cuervo Bedoya **Doc**: 1020468289 
* Mateo Loaiza Agudelo **Doc**: 1000661457

### ¿Regresión o clasificación?

Es un problema de regresión, ya que se quiere predecir la calidad del vino a partir de determinadas variables.

### Tipo de regresión

Se considerá inicialmente un modelo de regresión lineal (LinearRegression). 

Se medirá su desempeño y posteriormente se comparará con el desempeño de otro modelos como el árbol de decisión (DecisionTreeRegressor) y el bosque aleatorio (RandomForestRegressor), con el fin de elegir el modelo y en base a éste generar conclusiones.

### Procedencia de los datos

La base de datos tomada de internet, específicamente, de kaggle.
* **Fuente** : https://www.kaggle.com/datasets/uciml/red-wine-quality-cortez-et-al-2009/code

#### Descripción de las variables 

* **Acidez fija**: la mayoría de los ácidos del vino son fijos o no volátiles (no se evaporan fácilmente)
* **Acidez volátil**: la cantidad de ácido acético en el vino, que en niveles demasiado altos puede dar lugar a un sabor desagradable a vinagre
* **Ácido cítrico**: el ácido cítrico, que se encuentra en pequeñas cantidades, puede añadir "frescura" y sabor a los vinos.
* **Azúcar residual**: la cantidad de azúcar que queda después de la fermentación, es raro encontrar vinos con menos de 1 gramo/litro y los vinos con más de 45 gramos/litro se consideran dulces
* **Cloruros**: la cantidad de sal en el vino
* **Dióxido de azufre libre**: la forma libre de SO2 existe en equilibrio entre el SO2 molecular (como gas disuelto) y el ion bisulfito; impide el crecimiento microbiano y la oxidación del vino
**Dióxido de azufre total**: cantidad de formas libres y ligadas de S02; en bajas concentraciones, el SO2 es casi indetectable en el vino, pero en concentraciones de SO2 libre superiores a 50 ppm, el SO2 se hace evidente en la nariz y el sabor del vino
* **Densidad**: la densidad se aproxima a la del agua en función del porcentaje de alcohol y del contenido de azúcar
* **pH**: describe lo ácido o básico que es un vino en una escala de 0 (muy ácido) a 14 (muy básico); la mayoría de los vinos se sitúan entre 3 y 4 en la escala de pH
* **Sulfatos**: un aditivo del vino que puede contribuir a los niveles de gas de dióxido de azufre (S02), que actúa como antimicrobiano y antioxidante
* **Alcohol**: el porcentaje de alcohol del vino
* **Calidad**: variable de salida (basada en datos sensoriales, puntuación entre 0 y 10) **Variable objetivo**

### Función objetivo

El modelo medelo a comtemplar es el siguiente 
$$ h(x)=\sum_{i=0}^{11} W_{i}X_{i}$$

donde $X_{0}=1$ y $W \in R^{12}$

Buen uso de la notación vectorial para la descripción de la modelación

### Exploración gráfica

In [ ]:
vino.hist(figsize=(10,14))
plt.show()

Con ayuda de los histogramas anteriores, se puede ver que los vinos tienen una acidez fija promedio de 7.5, también se registra una acidez volátil promedio de 0.5, se tiene además que la acidez cítrica de la mayoría de vinos está alrededor de 0.

*Prefiero hablar de se presenta mayoritariamente en vez de se presenta en promedio, dado que los histogramas nos dan frecuencias no promedios.*

Por otro lado, podemos notar la densiadad y el pH tienen una distribución aproximadamente normal y como era esperarse, la variable calidad tiene un comportamieto bastante característico, pues toma valores enteros donde promedio está entre 5 y 6, en general, se puede decir que la calidad de estos vinos no es muy buena, ya que un vino  se considera sobresaliente cuando tiene una puntuación mayor o igual a 6.5.
 

In [ ]:
vino.info()

Podemos ver que ninguna de las variables en el conjunto de datos tiene ausencia de información. Se maneja en cada caso 1599 registros donde todas las variables son de tipo flotante excepto la variable "quality" (calidad) que es de tipo entero.

In [ ]:
vino.rename(columns = {"fixed acidity":"acidez fija", "volatile acidity":"acidez volatil",\
                        "citric acid":" acido citrico","residual sugar":"azucar residual",\
                        "chlorides":"cloruros", "free sulfur dioxide":"dioxido de azufre libre",\
                        "total sulfur dioxide":" dioxido de azufre total", "density":"densidad",\
                        "pH":"pH", "sulphates":"sulfatos", " alcohol":" alcohol", "quality":"calidad"
                        }, inplace = True)

Renombrarmos las columnas en español para mejor manejo del dataset.

In [ ]:
vino.head() #Primeros 5 registros del dataset con el cambio de nombres a español

### Análisis de correlaciones

In [ ]:
vino.corr()["calidad"].sort_values(ascending=False)

En el resumen de correlaciones anterior podemos observar que las variables que mayor influencia lineal positiva tienen sobre la calidad del vino son:
* alcohol
* sulfatos
* ácido cítrico

con una influencia lineal negativa tenemos:
* densidad
* dióxido de azufre total
* acidez volátil

Dado que el espacio de atributos es pequeño, decidimos trabajar con todo el conjunto.

*Buena inferencia*

### Separación inicial del conjunto de datos

Se separan los datos tomando una partición de 80% para entrenamiento y 20% para testeo.

In [ ]:
from sklearn.model_selection import train_test_split 
vino_train, vino_test = train_test_split(vino, test_size=0.2, random_state=513)

### Función para automatizar imputación y estandarización de datos

In [ ]:
from pandas.core.common import standardize_mapping
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

pipeline_num=Pipeline([
                         ("imputar", SimpleImputer(strategy= "median")),
                         ("estandarizar", StandardScaler())
                         ])

*Note que para usted no es necesario imputar la clase SimpleImputer pues ya estableció que no hay datos faltantes*

In [ ]:
vino_train_num=vino_train.drop("calidad", axis = 1) #Base de datos de entrenamiento exceptuando la variable objetivo (calidad)
vino_labels = vino_train[["calidad"]] #[[]] para que v_labels sea 2D

El pipeline creado tiene el objetivo de imputar si hay ausencia de datos y de estandarizar bases de datos futuras.
Dado que el dataset no tiene datos faltantes no es necesario usar el pipeline, por tanto, se estandarizarán los datos de manera directa.

*Buena justificación del uso de SimpleImputer* Los felicito. 

### Estandarización de los datos

In [ ]:
stan_scal=StandardScaler()
vino_train_num_prep=stan_scal.fit_transform(vino_train_num)

*Si ya incluyó la estandarización en la automatización con el Pipeline, no es necesario hacerlo por fuera*

### Entrenamiento y selección de modelos

In [ ]:
#Liberías para selección de modelos
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import RandomForestRegressor

In [ ]:
#Instanciamos los tres modelos que consideramos
r_lineal = LinearRegression()  #Modelo de regresión de lineal
r_tree = DecisionTreeRegressor() #Regresión con árbol de decisión
r_forest = RandomForestRegressor() #Regresión con bosque aleatorio

In [ ]:
vino_labels 

In [ ]:
#Ajustamos los modelos propuestos
r_lineal.fit(vino_train_num_prep, vino_labels)
r_tree.fit(vino_train_num_prep, vino_labels)
r_forest.fit(vino_train_num_prep, vino_labels.values.ravel())

*Note que las etiquetas las debe convertir a arreglos de numpy aplanados para entenar el modelo RandomForestRegresor*

### Desempeño de los modelos

Para medir el desempeño de los modelos vamos a utilizar el criterio de validación cruzada.

In [ ]:
from sklearn.model_selection import cross_val_score #Criterio de validación cruzada

In [ ]:
%%time 
score_tree = np.sqrt(-cross_val_score(r_tree, vino_train_num_prep, vino_labels, cv = 13,\
                             scoring = 'neg_mean_squared_error'))

score_lineal = np.sqrt(-cross_val_score(r_lineal, vino_train_num_prep, vino_labels, cv = 13,\
                              scoring = 'neg_mean_squared_error'))

score_forest = np.sqrt(-cross_val_score(r_forest, vino_train_num_prep,\
                                        vino_labels.values.ravel(), cv = 13,\
                               scoring = 'neg_mean_squared_error'))

In [ ]:
score_lineal.mean(), score_tree.mean(), score_forest.mean()

Dados los promedios de las puntuaciones de los criterios de validación cruzada, se tiene que el bosque aleatorio es modelo que mejor desempeño tiene, no obstante, el modelo regresión lineal también podría ser bueno dado que la media de las puntaciones es muy similar a la del bosque aleatorio.

*Estoy de ecuerdo con su inferencia, ya que el mejor modelo en este caso es el que tenga menor promedio de error cuadrático*

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
param_grid_forest = [
            { 'max_features': [2, 4, 6, 8, 10], 'n_estimators': [3, 10, 30, 40, 50, 60]},
            {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
             ]

In [ ]:

grid_search_forest = GridSearchCV(r_forest, param_grid_forest, cv=10, \
                                  scoring='neg_mean_squared_error',\
                           return_train_score=True)



In [ ]:
%%time 

grid_search_forest.fit(vino_train_num_prep, vino_labels.values.ravel())

In [ ]:
grid_search_forest.best_params_


In [ ]:
mejor_modelo_forest = grid_search_forest.best_estimator_


*Note que el método `.score()` asociado a la clase randomForestRegressor retorna es el coeficiente de determinación R2 y esta no es la medida de desempeño que usted eligió en el entrenamiento*

In [ ]:
np.sqrt(-cross_val_score(mejor_modelo_forest, vino_train_num_prep, vino_labels.values.ravel(), \
                         cv = 10,\
                        scoring = 'neg_mean_squared_error')).mean() 

Podemos notar que el modelo arrojado con la grilla de parámetros, es decir, el modelo RandomForestRegressor(max_features=2, n_estimators=40) cumple con que es el mejor modelo posible del random forest, dado que obtuvimos un error promedio de 0.5928069248291751.

### Desempeño de los datos testear y predicciones

In [ ]:
vino_test.head()

In [ ]:
vino_test_num=vino_test.drop("calidad", axis = 1) #Base de datos de testeo exceptuando la variable objetivo (calidad)
vino_labels_test = vino_test[["calidad"]] #[[]] para que v_labels sea 2D

In [ ]:
stan_scal_test=StandardScaler()
vino_test_num_prep=stan_scal_test.fit_transform(vino_test_num)

In [ ]:
from sklearn.metrics import mean_squared_error 
vino_test_predicciones=mejor_modelo.predict(vino_test_num_prep)
vino_test_predicciones[:5]

In [ ]:
np.sqrt(mean_squared_error(vino_labels_test, vino_test_predicciones))

Estandarizando el conjunto de testeo y calculando las predicciones con estos datos, podemos notar que hay una predicción bastante aceptable ya que en las primeras 5 predicciones solo falló en una.

## Referencias  

* La clase RandomForestRegressor:  https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

* La clase DecisionTreeRegressor: https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html  